In [6]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import calendar
import datetime

# Beispiel-Daten erstellen
data = {
    'Datum': pd.date_range(start='2024-06-01', periods=30, freq='D'),
    'Aktuelle_Besucher': [80, 100, 95, 110, 120, 130, 140, 115, 125, 135, 145, 155, 165, 175, 185, 195, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330],
    'Erwartete_Besucher': [85, 105, 100, 115, 125, 135, 145, 120, 130, 140, 150, 160, 170, 180, 190, 200, 205, 215, 225, 235, 245, 255, 265, 275, 285, 295, 305, 315, 325, 335],
    'Wetter': ['Sonnig', 'Wolkig', 'Regen', 'Sonnig', 'Wolkig', 'Regen', 'Sonnig', 'Wolkig', 'Regen', 'Sonnig', 'Wolkig', 'Regen', 'Sonnig', 'Wolkig', 'Regen', 'Sonnig', 'Wolkig', 'Regen', 'Sonnig', 'Wolkig', 'Regen', 'Sonnig', 'Wolkig', 'Regen', 'Sonnig', 'Wolkig', 'Regen', 'Sonnig', 'Wolkig', 'Regen'],
    'Ferien/Feiertage': ['Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Ja', 'Ja', 'Ja', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Ja', 'Ja', 'Ja', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Nein', 'Ja']
}

df = pd.DataFrame(data)

# Berechnungen für die Felder
aktuelle_gesamtbesucher = df['Aktuelle_Besucher'].sum()
relative_besucherzahl = df['Aktuelle_Besucher'].mean()
erwartete_auslastung = np.mean(df['Erwartete_Besucher'] / df['Aktuelle_Besucher']) * 100
erwartete_relative_auslastung = (df['Erwartete_Besucher'].sum() / aktuelle_gesamtbesucher) * 100

# Kalender-Daten für Feiertage
feiertag_df = df[df['Ferien/Feiertage'] == 'Ja'].copy()
feiertag_df = feiertag_df[['Datum']]

feiertag_df['Gap'] = feiertag_df['Datum'].diff().dt.days.ne(1).cumsum()
ranges = feiertag_df.groupby('Gap')['Datum'].agg(['min', 'max', 'count'])

# Separate valid ranges (at least 3 days) and individual dates
valid_ranges = ranges[ranges['count'] >= 3]
individual_dates = feiertag_df[~feiertag_df['Gap'].isin(valid_ranges.index)]

# Combine valid ranges and individual dates into a single list
events = []
for index, row in valid_ranges.iterrows():
    events.append({
        'type': 'range',
        'start': row['min'],
        'end': row['max']
    })

for index, row in individual_dates.iterrows():
    events.append({
        'type': 'individual',
        'date': row['Datum']
    })

# Sort the combined list by date
events.sort(key=lambda x: x['start'] if x['type'] == 'range' else x['date'])

# Erstellen des Dashboards
app = dash.Dash(__name__)

# Layout des Dashboards mit erweiterten Stiloptionen
app.layout = html.Div(style={
    'backgroundColor': '#f0f2f5', 
    'padding': '20px', 
    'fontFamily': 'Arial, sans-serif'
}, children=[
    html.Div(style={
        'display': 'flex', 
        'justifyContent': 'space-between', 
        'marginBottom': '20px'
    }, children=[
        html.Div([
            html.H3("Gesamtbesucher", style={
                'textAlign': 'center', 
                'color': '#003366'
            }),
            html.P(f"{aktuelle_gesamtbesucher}", style={
                'textAlign': 'center', 
                'fontSize': '25px', 
                'color': '#003366',
                'fontWeight': 'bold'
            })
        ], style={
            'width': '22%', 
            'backgroundColor': '#ffffff', 
            'padding': '15px', 
            'borderRadius': '10px', 
            'boxShadow': '0px 4px 12px rgba(0,0,0,0.1)',
            'margin': '5px'
        }),
        
        html.Div([
            html.H3("Relative Besucherzahl", style={
                'textAlign': 'center', 
                'color': '#003366'
            }),
            html.P(f"{relative_besucherzahl:.2f}", style={
                'textAlign': 'center', 
                'fontSize': '25px', 
                'color': '#003366',
                'fontWeight': 'bold'
            })
        ], style={
            'width': '22%', 
            'backgroundColor': '#ffffff', 
            'padding': '15px', 
            'borderRadius': '10px', 
            'boxShadow': '0px 4px 12px rgba(0,0,0,0.1)',
            'margin': '5px'
        }),

        html.Div([
            html.H3("Erwartete Auslastung", style={
                'textAlign': 'center', 
                'color': '#003366'
            }),
            html.P(f"{erwartete_auslastung:.2f}%", style={
                'textAlign': 'center', 
                'fontSize': '25px', 
                'color': '#003366',
                'fontWeight': 'bold'
            })
        ], style={
            'width': '22%', 
            'backgroundColor': '#ffffff', 
            'padding': '15px', 
            'borderRadius': '10px', 
            'boxShadow': '0px 4px 12px rgba(0,0,0,0.1)',
            'margin': '5px'
        }),

        html.Div([
            html.H3("Erwartete relative Auslastung", style={
                'textAlign': 'center', 
                'color': '#003366'
            }),
            html.P(f"{erwartete_relative_auslastung:.2f}%", style={
                'textAlign': 'center', 
                'fontSize': '25px', 
                'color': '#003366',
                'fontWeight': 'bold'
            })
        ], style={
            'width': '22%', 
            'backgroundColor': '#ffffff', 
            'padding': '15px', 
            'borderRadius': '10px', 
            'boxShadow': '0px 4px 12px rgba(0,0,0,0.1)',
            'margin': '5px'
        })
    ]),

    html.Div(style={
        'display': 'flex', 
        'justifyContent': 'space-between', 
        'gap': '20px' 
    }, children=[
        html.Div([
            html.H2("Besucherzahlen", style={
                'textAlign': 'center', 
                'color': '#003366'
            }),
            dcc.Graph(
                id='besucherzahlen',
                figure={
                    'data': [
                        go.Bar(
                            x=df['Datum'],
                            y=df['Aktuelle_Besucher'],
                            name='Aktuelle Besucher',
                            marker_color='#2e8b57' 
                        ),
                        go.Bar(
                            x=df['Datum'],
                            y=df['Erwartete_Besucher'],
                            name='Erwartete Besucher',
                            marker_color='#ffa500' 
                        )
                    ],
                    'layout': go.Layout(
                        title='Aktuelle und erwartete Besucherzahlen',
                        barmode='group',  # Balken nebeneinander gruppieren
                        plot_bgcolor='white',
                        paper_bgcolor='#f0f2f5',
                        font=dict(color='#003366'),
                        titlefont=dict(size=20),
                        xaxis=dict(
                            title='Datum',
                            titlefont=dict(size=18)
                        ),
                        yaxis=dict(
                            title='Anzahl der Besucher',
                            titlefont=dict(size=18)
                        ),
                        height=400  
                    )
                }
            )
        ], style={
            'width': '65%', 
            'backgroundColor': '#ffffff', 
            'padding': '20px', 
            'borderRadius': '10px', 
            'boxShadow': '0px 4px 12px rgba(0,0,0,0.1)',
            'marginBottom': '20px'
        }),
        
        html.Div(style={'width': '30%'}, children=[
            html.Div([
                html.H2("Wettervorhersage", style={
                    'textAlign': 'center', 
                    'color': '#003366'
                }),
                dcc.Graph(
                    id='wetter',
                    figure={
                        'data': [
                            go.Scatter(
                                x=df['Datum'],
                                y=df['Wetter'],
                                mode='lines+markers', 
                                name='Wetter',
                                line=dict(color='#4169e1')  
                            )
                        ],
                        'layout': go.Layout(
                            title='Wettervorhersage',
                            plot_bgcolor='white',
                            paper_bgcolor='#f0f2f5',
                            font=dict(color='#003366'),
                            titlefont=dict(size=20),
                            xaxis=dict(
                                title='Datum',
                                titlefont=dict(size=18)
                            ),
                            yaxis=dict(
                                title='Wetter',
                                titlefont=dict(size=18)
                            ),
                            height=300  
                        )
                    }
                )
            ], style={
                'marginBottom': '20px', 
                'backgroundColor': '#ffffff', 
                'padding': '20px', 
                'borderRadius': '10px', 
                'boxShadow': '0px 4px 12px rgba(0,0,0,0.1)'
            }),

            html.Div([
                html.H2("Feiertagskalender", style={
                    'textAlign': 'center',
                    'color': '#003366'
                }),
                html.Ul(
                    [
                        html.Li(
                            f"{event['start'].strftime('%Y-%m-%d')} bis {event['end'].strftime('%Y-%m-%d')}" if event['type'] == 'range' else event['date'].strftime('%Y-%m-%d'),
                            style={
                                'color': '#dc143c',
                                'fontSize': '18px'
                            }
                        ) for event in events
                    ],
                    style={
                        'listStyleType': 'none',
                        'padding': '0'
                    }
                )
            ], style={
                'backgroundColor': '#ffffff', 
                'padding': '20px', 
                'borderRadius': '10px', 
                'boxShadow': '0px 4px 12px rgba(0,0,0,0.1)'
            })
        ])
    ])
])

if __name__ == '__main__':
    app.run(host='localhost', port=8050)